In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

# Define function to generate data
def generate_data(likelihood_mean, likelihood_std, data_size):
    return np.random.normal(likelihood_mean, likelihood_std, data_size)

# Define function to get the prior distribution
def get_prior(prior_type, param1, param2):
    if prior_type == 'Gaussian':
        return stats.norm(param1, param2)
    elif prior_type == 'Poisson':
        return stats.poisson(param1)
    elif prior_type == 'Uniform':
        return stats.uniform(param1, param2)
    elif prior_type == 'Bernoulli':
        return stats.bernoulli(param1)
    else:
        raise ValueError("Unsupported prior distribution type")

# Define Bayesian update function
def bayesian_update(prior_type, param1, param2, likelihood_mean, likelihood_std, data_size):
    data = generate_data(likelihood_mean, likelihood_std, data_size)
    prior = get_prior(prior_type, param1, param2)
    
    if prior_type == 'Gaussian':
        prior_mean = param1
        prior_std = param2
        posterior_mean = (prior_mean / (prior_std ** 2) + np.sum(data) / (likelihood_std ** 2)) / (1 / (prior_std ** 2) + len(data) / (likelihood_std ** 2))
        posterior_std = np.sqrt(1 / (1 / (prior_std ** 2) + len(data) / (likelihood_std ** 2)))
        posterior = stats.norm(posterior_mean, posterior_std)
    elif prior_type == 'Poisson':
        posterior_mean = (param1 + np.sum(data)) / (1 + len(data))
        posterior = stats.poisson(posterior_mean)
    elif prior_type == 'Uniform':
        a = param1
        b = param1 + param2
        posterior_mean = (np.mean(data) + a + b) / 3
        posterior = stats.uniform(a, b - a)
    elif prior_type == 'Bernoulli':
        p = (param1 + np.sum(data)) / (1 + len(data))
        posterior = stats.bernoulli(p)
        posterior_mean = posterior.mean()
    else:
        raise ValueError("Unsupported prior distribution type")

    # Plotting
    x = np.linspace(-10, 10, 1000)
    plt.figure(figsize=(10, 6))
    if prior_type in ['Gaussian', 'Uniform']:
        plt.plot(x, prior.pdf(x), label='Prior', color='blue')
        plt.plot(x, posterior.pdf(x), label='Posterior', color='red')
    elif prior_type == 'Poisson':
        plt.vlines(x, 0, prior.pmf(x), label='Prior', color='blue')
        plt.vlines(x, 0, posterior.pmf(x), label='Posterior', color='red')
    elif prior_type == 'Bernoulli':
        plt.vlines([0, 1], 0, prior.pmf([0, 1]), label='Prior', color='blue')
        plt.vlines([0, 1], 0, posterior.pmf([0, 1]), label='Posterior', color='red')
    plt.hist(data, bins=30, density=True, alpha=0.5, label='Data', color='gray')
    plt.legend()
    plt.title('Bayesian Update')
    plt.xlabel('Parameter value')
    plt.ylabel('Density')
    plt.show()

    return posterior_mean

# Create widgets
prior_type_dropdown = widgets.Dropdown(options=['Gaussian', 'Poisson', 'Uniform', 'Bernoulli'], description='Prior Type:')
param1_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Param 1:')
param2_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Param 2:')
likelihood_mean_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Likelihood Mean:')
likelihood_std_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Likelihood Std:')
data_size_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Data Size:')

# Define a function to update the plot based on widget values
def update_plot(prior_type, param1, param2, likelihood_mean, likelihood_std, data_size):
    bayesian_update(prior_type, param1, param2, likelihood_mean, likelihood_std, data_size)

# Create interactive plot
interactive_plot = interactive(update_plot, prior_type=prior_type_dropdown, param1=param1_slider,
                               param2=param2_slider, likelihood_mean=likelihood_mean_slider,
                               likelihood_std=likelihood_std_slider, data_size=data_size_slider)

# Display the interactive plot
display(interactive_plot)


interactive(children=(Dropdown(description='Prior Type:', options=('Gaussian', 'Poisson', 'Uniform', 'Bernoull…